<div class="alert alert-danger">

<h1>Take notice!</h1>
<ul>
    <li>This class will be recorded</li>
</ul>
    
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Take-notice!" data-toc-modified-id="Take-notice!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Take notice!</a></span></li><li><span><a href="#Publishing-your-maps-and-charts-in-github-pages" data-toc-modified-id="Publishing-your-maps-and-charts-in-github-pages-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Publishing your maps and charts in github pages</a></span><ul class="toc-item"><li><span><a href="#Enable-github-pages" data-toc-modified-id="Enable-github-pages-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Enable github pages</a></span></li><li><span><a href="#Get-arrest-data" data-toc-modified-id="Get-arrest-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get arrest data</a></span></li><li><span><a href="#Save-interactive-elements-as-html-pages" data-toc-modified-id="Save-interactive-elements-as-html-pages-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Save interactive elements as html pages</a></span><ul class="toc-item"><li><span><a href="#Folium" data-toc-modified-id="Folium-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Folium</a></span></li><li><span><a href="#Kepler" data-toc-modified-id="Kepler-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Kepler</a></span></li><li><span><a href="#Plotly" data-toc-modified-id="Plotly-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Plotly</a></span></li></ul></li><li><span><a href="#Add-to-github-pages" data-toc-modified-id="Add-to-github-pages-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Add to github pages</a></span></li><li><span><a href="#Publish-to-ESRI-storymaps" data-toc-modified-id="Publish-to-ESRI-storymaps-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Publish to ESRI storymaps</a></span></li></ul></li></ul></div>

# Publishing your maps and charts in github pages
Github pages enables you to publish content in your repo that are in html format. This includes html files generated from folium maps, kepler maps, and interactive plotly charts.

## Enable github pages

- follow the steps in [this tutorial](https://guides.github.com/features/pages/) to enable github pages

## Get arrest data

In [ ]:
# to import data from LA Data portal
from sodapy import Socrata
import pandas as pd
import geopandas as gpd

# connect to the data portal
client = Socrata("data.lacity.org", None)

results = client.get("amvf-fr72", 
                     limit=50000,
                     where = "arst_date between '2021-09-01T00:00:00' and '2021-09-20T00:00:00'",
                     order='arst_date desc')

# Convert to pandas DataFrame
arrests = pd.DataFrame.from_records(results)

In [ ]:
arrests.shape

In [ ]:
# drop the unmapped rows
arrests.drop(arrests[arrests.lon==0].index,inplace=True)

In [ ]:
# convert lat/lon to floats
arrests.lon = arrests.lon.astype('float')
arrests.lat = arrests.lat.astype('float')

In [ ]:
arrests = arrests[['arst_date','grp_description','sex_cd','descent_cd','lat','lon']]

In [ ]:
arrests.head()

## Save interactive elements as html pages

For interactive visualizations, you can save them out as html files from within the Jupyter Notebook environment.

### Folium

In [ ]:
# import folium, a leaflet python library
import folium
from folium.plugins import MarkerCluster

In [ ]:
# create a list of lat/lon pair for UCLA
ucla = [34.0721237,-118.4440685]
m = folium.Map(location=ucla, zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

for index,row in arrests.iterrows():
    folium.Marker(
        location=[row.lat,row.lon],
        popup=row.grp_description,
        icon=folium.Icon(color="red", icon="ok-sign"),
    ).add_to(marker_cluster)

m

In [ ]:
# save as html file
m.save('ucla.html')

### Kepler

We create an interactive kepler map. The config variable below is the result of working within kepler to define the parameters of the map. This is just a time-saver to allow us to create a finished product to export.

In [ ]:
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": [
            "arrests"
          ],
          "id": "sreoedtdg",
          "name": [
            "arst_date"
          ],
          "type": "timeRange",
          "value": [
            1598918400000,
            1601078400000
          ],
          "enlarged": True,
          "plotType": "histogram",
          "yAxis": None
        }
      ],
      "layers": [
        {
          "id": "y8t676q",
          "type": "grid",
          "config": {
            "dataId": "arrests",
            "label": "Point",
            "color": [
              34,
              63,
              154
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "worldUnitSize": 0.5,
              "colorRange": {
                "name": "ColorBrewer RdYlGn-6",
                "type": "diverging",
                "category": "ColorBrewer",
                "colors": [
                  "#1a9850",
                  "#91cf60",
                  "#d9ef8b",
                  "#fee08b",
                  "#fc8d59",
                  "#d73027"
                ],
                "reversed": True
              },
              "coverage": 1,
              "sizeRange": [
                0,
                500
              ],
              "percentile": [
                0,
                100
              ],
              "elevationPercentile": [
                0,
                100
              ],
              "elevationScale": 20.9,
              "colorAggregation": "count",
              "sizeAggregation": "count",
              "enable3d": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantize",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "arrests": [
              {
                "name": "arst_date",
                "format": None
              },
              {
                "name": "grp_description",
                "format": None
              },
              {
                "name": "sex_cd",
                "format": None
              },
              {
                "name": "descent_cd",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 24,
      "dragRotate": True,
      "latitude": 33.837184166330836,
      "longitude": -118.46478962372794,
      "pitch": 50,
      "zoom": 9,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [ ]:
# kepler map of arrests with config parameters assigned
from keplergl import KeplerGl
map = KeplerGl(height=600, width=800, data={'arrests':arrests},config=config)
map

In [ ]:
map.save_to_html(file_name='arrests_kepler.html',read_only=True)

### Plotly


In [ ]:
import plotly.express as px

In [ ]:
# ok, group by date and charge, and let's get a count for each
arrests_by_type=arrests.groupby(['arst_date','grp_description']).size().reset_index(name='count')

In [ ]:
# make a stacked bar chart
fig = px.bar(arrests_by_type,
           x='arst_date',
           y='count',
           color='grp_description', # this creates the "stack",
           labels={'arst_date':'Arrest Date','count':'Number of arrests','grp_description':'Charge type'},
           title = 'September 2021 LAPD Arrests by Charge Type'
      )
fig.show()

In [ ]:
fig.write_html("arrests_stacked_plotly.html")

In [ ]:
# map it
fig = px.scatter_mapbox(arrests, 
                        lat="lat", 
                        lon="lon", 
                        color="descent_cd",
                        labels={'descent_cd':'Race of person arrested'}
                       )
fig.update_layout(mapbox_style="carto-positron")
fig.show()

In [ ]:
fig.write_html("arrests_map_plotly.html")

## Add to github pages

1. Make sure you have enabled github pages
1. Download the html file to your local computer
1. Upload the html file to your github repo
  - Make sure to note the folder where you upload to.
1. The path to your html file will be:


https://***your_github_username***.github.io/***name_of_your_repo***/***folder_path***(if not in root)/***your_html_file.html***

For example, to access the files created in this notebook from my repo, the links would be:


- https://yohman.github.io/21F-UP206A/Weeks/Week10/ucla.html
- https://yohman.github.io/21F-UP206A/Weeks/Week10/arrests_kepler.html
- https://yohman.github.io/21F-UP206A/Weeks/Week10/arrests_stacked_plotly.html
- https://yohman.github.io/21F-UP206A/Weeks/Week10/arrests_map_plotly.html


## Publish to ESRI storymaps

Example story map with embedded interactive Jupyter Notebook visualizations.

- https://arcg.is/1n0vGi